In [71]:
import numpy as np
import pandas as pd
import pickle
import time

In [72]:
# Path to the pickle files
PKL_DIR = 'Data/pickle'
CSV_DIR = 'Data'

# File names
train_interpolated = '/train_withInterpolations.pkl'
test_interpolated = '/test_withleak.pkl'
train_with_dates ='/train_withDateFeatures_without17304.pkl'
test_with_dates ='/test_withDateFeatures_without17304.pkl'
train_with_o ='/train2_otype.pkl'
test_with_o ='/test2_otype.pkl'
train_reduced = '/train_features_reduced_17304_non_cont.csv'
train_red_output = '/act_train_output.csv'
test_reduced = '/test_features_reduced_17304_non_cont.csv'

In [73]:
# Read all
train_data_i = pd.read_pickle(PKL_DIR+train_interpolated)
test_data_i = pd.read_pickle(PKL_DIR+test_interpolated)

In [74]:
train_data_d = pd.read_pickle(PKL_DIR+train_with_dates)
test_data_d = pd.read_pickle(PKL_DIR+test_with_dates)

In [75]:
train_data_o = pd.read_pickle(PKL_DIR+train_with_o)
test_data_o = pd.read_pickle(PKL_DIR+test_with_o)

In [76]:
# Read the train data set
train_data=pd.read_csv(CSV_DIR+train_reduced,parse_dates=["date","people_date"])

In [77]:
test_data=pd.read_csv(CSV_DIR+test_reduced,parse_dates=["date","people_date"])

In [78]:
train_data_d.columns

Index([u'activity_category', u'activity_id', u'char_1', u'char_10', u'char_2',
       u'char_3', u'char_4', u'char_5', u'char_6', u'char_7', u'char_8',
       u'char_9', u'date', u'dayOfMonth', u'days', u'month', u'outcome',
       u'outcome_filled', u'outcome_filled_nona', u'outcome_leak',
       u'people_char_1', u'people_char_10', u'people_char_11',
       u'people_char_12', u'people_char_13', u'people_char_14',
       u'people_char_15', u'people_char_16', u'people_char_17',
       u'people_char_18', u'people_char_19', u'people_char_2',
       u'people_char_20', u'people_char_21', u'people_char_22',
       u'people_char_23', u'people_char_24', u'people_char_25',
       u'people_char_26', u'people_char_27', u'people_char_28',
       u'people_char_29', u'people_char_3', u'people_char_30',
       u'people_char_31', u'people_char_32', u'people_char_33',
       u'people_char_34', u'people_char_35', u'people_char_36',
       u'people_char_37', u'people_char_38', u'people_char_4',
       u

In [79]:
Extra_date_columns = list(set(train_data_d.columns).difference(set(train_data.columns)))

In [80]:
Extra_outcome_columns = list(set(train_data_i.columns).difference(set(train_data.columns)))

In [81]:
Extra_o_columns = list(set(train_data_o.columns).difference(set(train_data.columns)))

In [82]:
list(set(train_data_o.columns).difference(set(train_data_d.columns)))

['gp_all0', 'gp_mixed', 'gp_all1']

In [83]:
train_data_o.shape

(1563770, 84)

In [84]:
train_data_d.shape

(1563770, 81)

In [85]:
train_data.shape

(1398166, 68)

In [86]:
train_data_i.shape

(1398166, 72)

In [87]:
test_data.shape

(333083, 68)

In [88]:
# Extra_o_columns

In [89]:
merged_all_features = pd.merge(train_data,train_data_o[['activity_id']+Extra_o_columns],on='activity_id',how='left')

In [90]:
train = merged_all_features

In [91]:
test = pd.merge(test_data,test_data_o[['activity_id']+Extra_o_columns],on='activity_id',how='left')

In [92]:
print train.shape,test.shape

(1398166, 84) (333083, 84)


In [93]:
from sklearn.preprocessing import OneHotEncoder

In [94]:
# Function to change labels of categories to one-hot encoding using scikit's OneHot Encoding
# pd.get_dummies(df) does the same, provides sweet header's as well but it it not fast enough, kill's memory
def category_to_one_hot(dataset,non_feature,continuous_feature):
    ds = dataset.drop(non_feature,axis=1)
    boolean_column = []
    counter=0
    for column in ds.columns:
        if column not in continuous_feature:
            boolean_column.append(counter)
        counter += 1
    # boolean_colum is not the column name but index
    print ("Done filtering columns...")
    grd_enc = OneHotEncoder(categorical_features=boolean_column)
    encoded_arr=grd_enc.fit_transform(ds)
    return encoded_arr

In [95]:
set(train.columns).difference(set(train_data.columns))

{'activities_per_group_date',
 'adays_till_activity',
 'diff_date',
 'gp_all0',
 'gp_all1',
 'gp_mixed',
 'next_outcome',
 'outcome',
 'outcome_filled',
 'outcome_filled_nona',
 'outcome_leak',
 'pdays_till_activity',
 'people_per_group',
 'people_per_group_date',
 'prev_outcome',
 'worked_for_day'}

In [96]:
# Non feature
NON_FEATURE=['activity_id','people_id','date','people_date','char_10','outcome', 'outcome_leak','outcome_filled']

# Categorical data that is only label encoded
CATEGORICAL_DATA = ['people_char_1', 'people_char_2','people_group_1',
                    'people_char_3', 'people_char_4', 'people_char_5',
                    'people_char_6', 'people_char_7', 'people_char_8',
                    'people_char_9', 'activity_category',
                    'char_1', 'char_2', 'char_3', 'char_4', 'char_5', 'char_6',
                    'char_7', 'char_8', 'char_9']

# Already in a one-hot encoded form
CATEGORICAL_BINARY = ['people_char_10', 'people_char_11', 'people_char_12',
                      'people_char_13', 'people_char_14', 'people_char_15',
                      'people_char_16', 'people_char_17', 'people_char_18',
                      'people_char_19', 'people_char_20', 'people_char_21',
                      'people_char_22', 'people_char_23', 'people_char_24',
                      'people_char_25', 'people_char_26', 'people_char_27',
                      'people_char_28', 'people_char_29', 'people_char_30',
                      'people_char_31', 'people_char_32', 'people_char_33',
                      'people_char_34', 'people_char_35', 'people_char_36',
                      'people_char_37','weekend' ]

# Continuous categories
CONT = ['people_days', 'days',
      'people_month',  'month', 
      'people_quarter', 'quarter',
      'people_week', 'week',
      'people_dayOfMonth', 'dayOfMonth',
      'people_year', 'year',
      'people_char_38','activities_per_group_date',
      'adays_till_activity',
      'diff_date',
      'next_outcome',
      'outcome_filled_nona',
      'pdays_till_activity',
      'people_per_group',
      'people_per_group_date',
      'prev_outcome',
       'worked_for_day',
       'gp_all0',
      'gp_all1',
      'gp_mixed']

In [97]:
## Replacing NaN's ##
test['next_outcome'] = test['next_outcome'].fillna(-1)
test ['prev_outcome'] = test ['prev_outcome'].fillna(-1)

In [98]:
train['next_outcome'] = train['next_outcome'].fillna(-1)
train ['prev_outcome'] = train['prev_outcome'].fillna(-1)

In [99]:
for column in CONT:
    if train[column].isnull().sum() > 0:
        print (column)

In [100]:
for column in CONT:
    if test[column].isnull().sum() > 0:
        print (column)

In [101]:
# people_char_3, char_5
set(train['people_char_3'].unique()).difference(set(test['people_char_3'].unique()))

{99999}

In [102]:
set(train['char_5'].unique()).difference(set(test['char_5'].unique()))

{99999}

In [103]:
# Introducing a dummy row in test to ensure consistent ohe, not relevant to logic
a_row = test.iloc[0]

In [104]:
a_row['activity_id']='act_0'
a_row['char_5']=99999
a_row['people_char_3']=99999

/Users/aakash.gupta/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/aakash.gupta/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/aakash.gupta/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [105]:
test = test.append(a_row)

In [106]:
# # UNCOMMENT AND CHECK THIS ONCE, the total number of columns should match for each print statement
# # Function to one hot encode all values 
# for col in CATEGORICAL_DATA:
#     start=time.time()
#     arr=category_to_one_hot(train[CONT+CATEGORICAL_BINARY+NON_FEATURE+[col]],NON_FEATURE,CONT+CATEGORICAL_BINARY)
# #     test_data_df.drop('outcome',inplace=True,errors='ignore')
    
#     arr_b=category_to_one_hot(test[CONT+CATEGORICAL_BINARY+NON_FEATURE+[col]],NON_FEATURE,CONT+CATEGORICAL_BINARY)
#     end=time.time()
#     print(end-start)
#     print (arr.shape)
#     print (arr_b.shape)
#     print ("-----"+str(col)+"-----")

In [107]:
train.to_pickle(PKL_DIR+'/train_merged2_without17304.pkl')
test.to_pickle(PKL_DIR+'/test_merged2_without17304.pkl')